## Setup Database for Hive Metastore

Let us go ahead and setup Postgres database for Hive Metastore. We can setup Postgres database using Docker which was setup earlier on this Ubuntu 18.04 based server.

* We can setup single session database for Hive Metastore. However, it have limitations and might get easily corrupted.
* Once we setup Postgres database, we can configure Hive Metastore using the Postgres database.
* Make sure the user is member of docker group. If the user is just added, make sure to restart the session (exit and relogin using SSH).

```shell
id ${USER}
sudo usermod -aG docker ${USER}
```

* Use below commands to setup fresh postgres database. It is named as **cluster_util_db** as we will be using the same database for other utilities as well.

```shell
docker pull postgres

docker create \
    --name cluster_util_db \
    -p 6432:5432 \
    -e POSTGRES_PASSWORD=itversity \
    postgres

docker start cluster_util_db

docker logs -f cluster_util_db # Use Ctrl+C to come out of the logs
```

* You can validate connectivity to Postgres database by running this command.

```shell
docker exec \
    -it cluster_util_db \
    psql -U postgres

# You can use \q to come out of postgres.
```

* Let us create a database for Hive Metastore using Postgres Database Server just created.

```shell
docker exec \
    -it cluster_util_db \
    psql -U postgres
```

```sql
CREATE DATABASE metastore;
CREATE USER hive WITH ENCRYPTED PASSWORD 'itversity';
GRANT ALL ON DATABASE metastore TO hive;

\l
\q
```

* Let us also setup client on the host to validate connectivity to the postgres database running as part of docker.

```shell
sudo apt install postgresql-client -y
```

* We can use this command to validate connectivity to postgres database running in Docker container from the host.

```shell
telnet localhost 6432 
# Use Ctrl+] to go to prompt and then use quit to come out of telnet

psql -h localhost \
    -p 6432 \
    -d metastore \
    -U hive \
    -W    
```

* Let us also setup below environment variables as part of **.profile**. Make sure to restart the session or run the profile after setting up the below environment variables.

```shell
export HIVE_HOME=/opt/hive
export PATH=$PATH:${HIVE_HOME}/bin
```